#### Chat with CSV file Using GeminiPro LLM

In [8]:
import pandas as pd

df = pd.read_csv("1.csv")
print(df.shape)
print(df.columns.tolist())

(124213, 4)
['Inspection ID', 'Inspection date', 'Part detected', 'Vehicle ID']


In [9]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///analysis.db")
df.to_sql("analysis", engine, index=False)

124213

In [12]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
#db.run("SELECT * FROM analysis WHERE Inspection date > 13 Apr 2023;")

sqlite
['analysis']


In [35]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key='API_KEY')
agent_executor = create_sql_agent(llm, db=db, agent_type='zero-shot-react-description')



In [36]:
agent_executor.invoke({"input": "what are most top 3 detected parts and their counts "})

{'input': 'what are most top 3 detected parts and their counts ',
 'output': "[('Right front door', 18194), ('Right fender', 16203), ('Front bumper', 16110)]"}

In [ ]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain_community.agent_toolkits import create_sql_agent
from langchain_google_genai import ChatGoogleGenerativeAI


df = pd.read_csv("1.csv")

engine = create_engine("sqlite:///analysis.db")
df.to_sql("analysis", engine, index=False)
db = SQLDatabase(engine=engine)

llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=API_KEY)
agent_executor = create_sql_agent(llm, db=db, agent_type='zero-shot-react-description')

agent_executor.invoke({"input": "what are most top 3 detected parts and their counts "})


In [4]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch(share=True)   

Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [3]:
import gradio as gr
import pandas as pd
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain_community.agent_toolkits import create_sql_agent
from langchain_google_genai import ChatGoogleGenerativeAI

def process_query(csv_file, api_key, user_query):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file.name)
    
    # Create the SQLite database and load the DataFrame into it
    engine = create_engine("sqlite:///analysis.db")
    df.to_sql("analysis", engine, index=False, if_exists="replace")
    db = SQLDatabase(engine=engine)
    
    # Initialize the Google Generative AI model with the provided API key
    llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
    
    # Create the SQL agent
    agent_executor = create_sql_agent(llm, db=db, agent_type='zero-shot-react-description')
    
    # Execute the user's query
    result = agent_executor.invoke({"input": user_query})
    
    # Return the result
    return result['output']

# Create the Gradio interface
iface = gr.Interface(
    fn=process_query,
    inputs=[
        gr.File(label="Upload your CSV file"),
        gr.Textbox(label="Enter your Google Generative AI API key"),
        gr.Textbox(label="Enter your query")
    ],
    outputs="text",
    title="Data Analysis with Google Generative AI",
    description="Upload a CSV file, provide your Google Generative AI API key, and ask questions about the data."
)

# Launch the app
iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7862

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
